# CDJ Project Social Security

The purpose of this file is to create the Sankey diagram based on real proportions from our data. This is for figure 2.

Data from https://www.ssa.gov/oact/STATS/table3c3.html (df_fica) and https://www.ssa.gov/oact/progdata/icp.html (df_ben)

In [25]:
#Import Packages

import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
#Import FICA Tax Breakdown
df_fica = pd.read_excel("OASDI_FICA_Breakdown.xlsx") #extract excel
df_fica = df_fica.T #transpose
df_fica.columns = df_fica.iloc[0] 
df_fica.drop(index = "Program and Source", inplace = True) #set index

In [29]:
#Create columns to plot.

df_fica["tot"] = df_fica["OASI_Employer"]+df_fica["OASI_Employee"]+df_fica["OASI_Self-employed"]+df_fica["DI_Employer"]+df_fica["DI_Employee"]+df_fica["DI_Self-employed"]
df_fica["Employer_Employee"] = (df_fica["OASI_Employer"]+df_fica["OASI_Employee"]+df_fica["DI_Employer"]+df_fica["DI_Employee"])/df_fica["tot"]
df_fica["self_employ"] = (df_fica["OASI_Self-employed"]+df_fica["DI_Self-employed"])/df_fica["tot"]

In [31]:
#Import Beneficiary Data 

df_ben = pd.read_excel("SS_Data_by_Type_of_Beneficiary2.xlsx") #extract excel
df_ben.rename(columns={ df_ben.columns[0]: "Year" }, inplace = True) #rename year column

In [33]:
#INTERMEDIATE DATA PROCESSING. CAN IGNORE

df_ben_year = df_ben.groupby(df_ben['Year'].dt.year).apply(lambda x: x.iloc[:, 1:].sum())
def extract_beneficiary_type(specified_beneficiary):
    x = df_ben_year.columns.str.startswith(specified_beneficiary) #gets the columns in an index with T and F
    #x[0] = True #makes the date column from False to True
    df_return = df_ben_year[df_ben_year.columns[x]]
    return df_return

beneficiary_types_ls = [
    
    "Retired_Worker", #0 - ret work
    "Spouse_of_Retired_Worker", #1 - ret work
    "Child_Of_Retired_Worker", #2 - ret work
    "Child_Of_Deceased_Worker", #surv - 3
    "Aged_Widower",#surv
    "Young_Widower_With_Child_In_Care",#surv
    "Disabled_Widower",#surv
    "Parent_Of_Deceased_Worker",#surv -7
    "Disabled_Worker", #dis - 8
    "Spouse_Of_Disabled_Worker",
    "Child_Of_Disabled_Worker" #dis - 10
]


In [43]:
#INTERMEDIATE DATA PROCESSING. CAN IGNORE

retired_workers_category = extract_beneficiary_type(beneficiary_types_ls[0]).sum(axis = 1) + \
    extract_beneficiary_type(beneficiary_types_ls[1]).sum(axis = 1) + \
    extract_beneficiary_type(beneficiary_types_ls[2]).sum(axis = 1)

survivors_category = extract_beneficiary_type(beneficiary_types_ls[3]).sum(axis = 1) + \
    extract_beneficiary_type(beneficiary_types_ls[4]).sum(axis = 1) + \
    extract_beneficiary_type(beneficiary_types_ls[5]).sum(axis = 1) + \
    extract_beneficiary_type(beneficiary_types_ls[6]).sum(axis = 1) + \
    extract_beneficiary_type(beneficiary_types_ls[7]).sum(axis = 1)

disabled_category = extract_beneficiary_type(beneficiary_types_ls[8]).sum(axis = 1) + \
    extract_beneficiary_type(beneficiary_types_ls[9]).sum(axis = 1) + \
    extract_beneficiary_type(beneficiary_types_ls[10]).sum(axis = 1)

tot = df_ben_year[:].sum(axis = 1)

df_ben_year["Retired_prop"] = retired_workers_category/tot #creates column of retired worker proportion compared to all beneficiaries
df_ben_year["Survivors_prop"] = survivors_category/tot #creates column of survivors proportion compared to all beneficiaries
df_ben_year["Disabled_prop"] = disabled_category/tot #creates column of disabled proportion compared to all beneficiaries

#each of the above df has proportions from year 1967 to 2024 inclusive

In [55]:
# Define the nodes (with unique flow relationships)
node_labels = [
    "Employer + Employee 6.2% FICA",  # 0
    "Self-employed 14.2% FICA",       # 1
    "Social Security Trust Fund",     # 2
    "Retirement (OASI)",               # 3
    "Retired Workers",                 # 4
    "Normal Retirement (≈66)",       # 5
    "Early Retirement (< 66)",  # 6
    "Delayed Retirement (> 70)",       # 7
    "Survivors",                      # 8
    "Disability Insurance (DI)"       # 9
]

# Define the links (flows between nodes)
link_sources = [
    0, 1,  # Employer + Employee FICA and Self-employed FICA -> Trust Fund
    2, 2,  # Trust Fund -> Retirement (OASI) and Disability Insurance (DI)
    3, 3,  # Retirement (OASI) -> Retired Workers and Survivors
    4, 4, 4  # Retired Workers -> FRA, Early FRA, Max
]

link_targets = [
    2, 2,  # Employer + Employee and Self-employed FICA -> Trust Fund
    3, 9,  # Trust Fund -> Retirement (OASI) and Disability Insurance (DI)
    4, 8,  # Retirement (OASI) -> Retired Workers and Survivors
    6, 5, 7  # Retired Workers -> FRA, Early FRA, Max
]

# Manually define the flow values (can be modified with your actual data)
link_values = [
    100*df_fica["Employer_Employee"].loc[2023], 100*df_fica["self_employ"].loc[2023],  # FICA contributions in 2023
    100*(df_ben_year["Retired_prop"].loc[2023]+df_ben_year["Survivors_prop"].loc[2023]), 100*df_ben_year["Disabled_prop"].loc[2023],    # Trust Fund flows
    100*(df_ben_year["Retired_prop"].loc[2023]+df_ben_year["Survivors_prop"].loc[2023])*(df_ben_year["Retired_prop"].loc[2023]), \
        100*(df_ben_year["Retired_prop"].loc[2023]+df_ben_year["Survivors_prop"].loc[2023]) -  100*(df_ben_year["Retired_prop"].loc[2023]+df_ben_year["Survivors_prop"].loc[2023])*(df_ben_year["Retired_prop"].loc[2023]),    # Retirement (OASI) flows
    68.1*(1-0.2), 68.1*0.14, 68.1*0.06 # Retired Workers flows
]

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=40,               # Increase space between nodes
        thickness=20,         # Adjust node thickness
        line=dict(color="black", width=0.5),
        label=node_labels,
        color="blue",
        # Adjust Y positions of nodes to avoid overlap and make the OASI branch go up
        y=[0.85, 0.85, 0.6, 0.75, 0.5, 0.35, 0.35, 0.35, 0.25, 0.2]  # Adjusted Y positions
    ),
    link=dict(
        source=link_sources,
        target=link_targets,
        value=link_values,
        color="gray"
    )
)])

# Update the layout to make the diagram taller and more readable
fig.update_layout(
    title_text="Social Security Trust Fund Money Inflows and Outflows",
    font_size=10,
    height=600,   # Increase height for vertical space
    width=1000,   # Adjust width for better node spacing
    showlegend=False  
)

fig.show()


In [53]:
#OLD GRAPH DO NOT USE


df_ben_year

plt.figure(figsize = (12,4))

plt.stackplot(range(1967, 2025),
              df_ben_year["Retired_prop"], 
              df_ben_year["Survivors_prop"], 
              df_ben_year["Disabled_prop"],
              labels=["Retired", "Survivors", "Disabled"])
plt.legend(loc="lower right")
plt.xlabel("Year")
plt.ylabel("Proportion")
plt.title("Proportions of Retired, Survivors, and Disabled from 1967 to 2024")
plt.show()